# Sudoku Generation

This notebook is used to create functions and use them to create various sudoku grids using sudoku shuffling techniques.

A valid sudoku table has following properties: (source: https://en.wikipedia.org/wiki/Sudoku_solving_algorithms )
1. contains 81 numbers (cells in a 9x9 grid with 9 boxes)
2. each box being the intersection of the first, middle, or last 3 rows, and the first, middle, or last 3 columns
3. each cell contains number from 1 to 9
4. each number can only occur once in each row, column and box

In sudoku, you can do following operations to keep the table valid:
1. swap row/column blocks
2. swap all occurances of one number with all occurances of another number
3. swap rows/columns in a block

By doing such operations, you can create many new sudoku tables that will eventually result in a different problems using one input. 

Let's now create a set of functions that will randomly apply the rules above on existing sudoku table.

In [ ]:
import random 

#swap all occurances of two numbers in a grid
def swap_numbers(grid, x, y):
    return grid.replace(x, 'x').replace(y, x).replace('x', y)

#swap rows within a block
def swap_rows(grid):
    #extract rows
    rows = [grid[i*9:(i+1)*9] for i in range(9)]
    
    #shuffle rows within blocks
    for i in range(0, 9, 3):
        block_rows = rows[i:i+3]
        random.shuffle(block_rows)
        rows[i:i+3] = block_rows
    
    #reconstruct grid
    return ''.join(rows)

#swap columns within a block
def swap_columns(grid):
    #extract columns
    columns = []
    for i in range(9):
        column = [int(grid[j*9 + i]) for j in range(9)]
        columns.append(column)
    
    #shuffle columns within blocks
    for i in range(0, 9, 3):
        block_columns = columns[i:i+3]
        random.shuffle(block_columns)
        columns[i:i+3] = block_columns
    
    #reconstruct grid
    return ''.join(''.join(str(columns[j][i]) for j in range(9)) for i in range(9))
